# Data Cleaning de la base scrappeada de HowLongToBeat.com

In [1]:
# Librerias necesarias
import pandas as pd
import numpy as np

## Arreglos realizados en Excel

 - Se separo el archivo en las columnas correspondientes
 - En las columnas con informacion de tiempos de completitud se modifico los casos en donde figuran horas fraccionarias (1/2 hours) por 0,5.
 - Por otro lado, en las mismas columnas, donde figuraba el tiempo en minutos, se transformo en su equivalente en horas (en formato float)
 - Los campos incompletos (en donde el scrapper no encontro informacion) se modificaron de ***"--"*** a nulos.
 - En los campos sin Rating (NR) se modificaron a nulos.
 - Se eliminaron los juegos con Type "DLC", "Expansion", etc. De esta manera solo con quedamos con jueos completos.


In [52]:
# Importo el archivo con la base de datos
df = pd.read_excel('C:/Users/Ramiro/Desktop/Digital House/Desafios/HLTB Scrapper/HowLongToBeat DB.xlsx')

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48157 entries, 0 to 48156
Data columns (total 15 columns):
Id               48157 non-null int64
Titulo           48157 non-null object
Main Story       15165 non-null float64
Main + Extras    9929 non-null float64
Completionist    11180 non-null float64
All Styles       18386 non-null float64
Rating           19973 non-null float64
Developers       27844 non-null object
Publishers       26637 non-null object
Playable On      16915 non-null object
Genres           27406 non-null object
NA               25709 non-null object
EU               10630 non-null object
JP               4751 non-null object
Updated          48007 non-null object
dtypes: float64(5), int64(1), object(9)
memory usage: 5.5+ MB


In [17]:
#Analizamos como esta compuesto el archivo
df.sample(10)

,Id,Titulo,Main Story,Main + Extras,Completionist,All Styles,Rating,Developers,Publishers,Playable On,Genres,Type,NA,EU,JP,Updated
38818,48509,Adventures Of Kok,NaN,NaN,NaN,NaN,NaN,Deep dark fantasy games,Illumicorp,NaN,Adventure,NaN,"July 31, 2017","July 31, 2017",NaN,9.5 Months Ago
19342,22903,Magnifico,0.783333,NaN,NaN,0.783333,70.0,Mixel,Matrix Games,PC,"Board Game, Strategy",NaN,NaN,NaN,NaN,1 Week Ago
11620,13231,PokÃ©mon Puzzle Collection,NaN,NaN,NaN,NaN,NaN,Jupiter,Nintendo,NaN,Puzzle,NaN,NaN,NaN,"November 20, 2001",1 Year Ago
21158,25088,Nicolas Eymerich: The Inquisitor - Book II: Th...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 Years Ago
11807,13503,Bruce Lee: Quest of the Dragon,5.500000,NaN,NaN,5.500000,50.0,Ronin Entertainment,Universal Interactive,Xbox,Beat 'em Up,NaN,"July 02, 2002","September 06, 2002",NaN,8.5 Months Ago
41355,51439,"Hunter, In Darkness",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11 Months Ago
48149,58860,Fast Rolling,NaN,NaN,NaN,NaN,NaN,FastRolling,Atriagames,NaN,NaN,NaN,"December 09, 2017","December 09, 2017",NaN,2.5 Weeks Ago
46284,56869,Cops and Robbers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.5 Months Ago
4509,4986,Killzone 3,7.000000,10.5,18.0,7.500000,77.0,Guerrilla Games,Sony Computer Entertainment,PlayStation 3,"First-Person, Shooter",NaN,"February 22, 2011","February 23, 2011",NaN,21 Hours Ago
1805,1988,Crashday,7.000000,7.5,8.0,7.500000,67.0,"Replay Studios, Moonbyte Games",Atari,PC,Racing,NaN,"December 20, 2006",NaN,NaN,1 Month Ago


El dataset esta compuesto por 48.157 observaciones con los siguientes campos:
 - __Id__: Id en la web de How Long To Beat
 - __Titulo__: Nombre del videojuego
 - __Main Story__: El tiempo necesario para completar la historia principal
 - __Main Story + Extras__: El tiempo necesario para completar la historia principal y los "sidequests"
 - __Completionist__: El tiempo necesario para completar el juego al 100%
 - __All Styles__: Un tiempo aproximado de los distintos estilos combinados
 - __Rating__: Score otorgado al juego
 - __Developer__: Lista de Desarrolladores
 - __Publishers__: Lista de compañias que publicaron el juego
 - __Playable On__: Consolas en las cuales esta disponible el juego
 - __Genres__: Generos a los cuales pertenece el juego
 - __NA__: Fecha de lanzamiento en Norteamerica
 - __EU__: Fecha de lanzamiento en Europa
 - __JP__: Fecha de lanzamiento en Japon
 - __Updated__: Ultima actualizacion de la informacion del juego

Como era de esperarse, del scrapping inicial hace falta hacer una limpieza para poder dejar listos los datos para el modelado.
Encontramos los siguientes problemas:
 - Las columnas de tiempos queremos redondearlas a dos decimales.
 - Queremos que el indice del df sea el numero de id del juego en la web para poder hacer un rastreo del mismo
 - Queremos separar las columnas de Genres, Playable On, Developers y Publishers en dummies para el modelo


In [53]:
#Arreglamos las columnas con ceros extra
df['Main Story'] = df['Main Story'].astype(float).round(2)
df['Main + Extras'] = df['Main + Extras'].astype(float).round(2)
df['Completionist'] = df['Completionist'].astype(float).round(2)
df['All Styles'] = df['All Styles'].astype(float).round(2)

In [4]:
#Seteamos el indice 
df = df.set_index(df['Id'])
df = df.drop(columns='Id')

In [27]:
df.head(10)

,Titulo,Main Story,Main + Extras,Completionist,All Styles,Rating,Developers,Publishers,Playable On,Genres,Type,NA,EU,JP,Updated
Id,,,,,,,,,,,,,,,
1,688(I) Hunter/Killer,9.5,24.0,50.5,15.5,NaN,Sonalysts,Electronic Arts,PC,Simulation,NaN,"July 04, 1997",NaN,NaN,1 Month Ago
2,Beyond Good & Evil 2,NaN,NaN,NaN,NaN,NaN,Ubisoft Montpellier,Ubisoft,NaN,"Action, Adventure",NaN,2016,NaN,NaN,3.5 Months Ago
10,Ico and Shadow of the Colossus Collection,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 Years Ago
16,Shadow of the Comet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 Years Ago
18,'Splosion Man,8.0,10.0,18.0,9.5,76.0,Twisted Pixel Games,Microsoft Games Studios,Xbox 360,"Action, Platformer",NaN,"July 22, 2009","July 22, 2009",NaN,21.5 Hours Ago
19,.hack//G.U. Vol. 1: Rebirth,20.5,26.5,38.0,24.5,80.0,CyberConnect2,Bandai,"PC, PlayStation 2, PlayStation 4","Action, Role-Playing",NaN,"October 24, 2006",NaN,"May 18, 2006",20.5 Hours Ago
20,.hack//G.U. Vol. 2: Reminisce,22.5,37.0,48.0,31.5,81.0,CyberConnect2,Bandai,"PlayStation 2, PlayStation 4","Action, Role-Playing",NaN,"May 08, 2007",NaN,"September 28, 2006",13 Hours Ago
21,.hack//G.U. Vol. 3: Redemption,23.5,35.5,52.5,30.0,79.0,CyberConnect2,Bandai,"PlayStation 2, PlayStation 4","Action, Role-Playing",NaN,"March 25, 2008",NaN,"January 18, 2007",19.5 Hours Ago
22,.hack//Infection,18.0,20.0,28.5,20.5,78.0,CyberConnect2,Bandai S.A.,PlayStation 2,"Action, role-playing",NaN,"February 11, 2003",NaN,NaN,20 Hours Ago


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48157 entries, 0 to 48156
Data columns (total 15 columns):
Id               48157 non-null int64
Titulo           48157 non-null object
Main Story       15165 non-null float64
Main + Extras    9929 non-null float64
Completionist    11180 non-null float64
All Styles       18386 non-null float64
Rating           19973 non-null float64
Developers       27844 non-null object
Publishers       26637 non-null object
Playable On      16915 non-null object
Genres           27406 non-null object
NA               25709 non-null object
EU               10630 non-null object
JP               4751 non-null object
Updated          48007 non-null object
dtypes: float64(5), int64(1), object(9)
memory usage: 5.5+ MB


# Modelado

El objetivo de este trabajo es poder crear un modelo predictivo, que a partir de la informacion de un juego, como cual fue su developer, publisher, genero, etc; pueda determinar un rating entre 0 y 100.<br>
Nos interesa especialmente que el modelo pueda acomodarse a valores faltantes en algunos de los campos, para lo cual vamos a utilizar XGBoost.<br>
Al mismo tiempo, vamos a plantear otro modelos de regresion para datasets completos y comparar la performance de ambos.

## Modelo para predecir tiempos??

In [15]:
#Separo en dos datasets para cada tipo de modelo
df_boosting = df.drop(columns=['NA','EU','JP','Updated'])
df_reg = df.drop(columns=['NA','EU','JP','Updated'])

In [16]:
#Eliminamos los nulos para los modelos de regresion mas simples
df_reg = df_reg.dropna(how='any')

In [19]:
#Como vamos a buscar predecir ratings, no nos sirven todas las observaciones que no tienen este campo completo.
df_boosting = df_boosting.dropna(subset = ['Rating'])


Para poder proveerle la data al modelo, necesitamos que las columnas que tienen mas de un valor (como Genres) sean transformadas en dummies. <br>
Para eso vamos a definir una funcion que itere sobre las columnas correspondientes, y extraiga todos los valores unicos, para luego crear una columna por cada uno de esos valores y completarla con 1 o 0 dependiendo si pertenece a esa clase o no.

### Generos

In [36]:
generos_con_num = [2042,3079,4160,4828,4884,4886,4887,5686,7371,7921,8127,8468,8469,8744,10369,10992,11381]


In [40]:
df = df.drop(df.index[[2042,3079,4160,4828,4884,4886,4887,5686,7371,7921,8127,8468,8469,8744,10369,10992,11381]])

In [71]:
#Lista de Generos
import itertools
unique_genres = df['Genres'].unique()
individual_genres = []
for genre in unique_genres:
    individual_genres.append(genre.split(','))

individual_genres = list(itertools.chain.from_iterable(individual_genres))
individual_genres = set(individual_genres)

individual_genres

{' 3-D',
 ' Action',
 ' Action Adventure',
 ' Action RPG',
 ' Action role-playing',
 ' Action-Adventure',
 ' Action-adventure',
 ' Adventure',
 ' Adventure game',
 ' Aerial combat',
 ' Arcade',
 ' Artillery',
 ' Battle Arena',
 " Beat 'em Up",
 " Beat 'em up",
 " Beat'em up",
 ' Block-breaker',
 ' Breakout',
 ' Car Combat',
 ' Card Game',
 ' Casual',
 ' City-Builder',
 ' City-Building',
 ' Combat Flight Simulator',
 ' Compilation',
 ' Crossover',
 ' Driving',
 ' Dungeon Crawl',
 ' Educational',
 ' Exercise',
 ' Family',
 ' Family Games',
 ' Fighting',
 ' First-Person',
 ' First-person shooter',
 ' Flight',
 ' Flight Combat',
 ' God game',
 ' Hack and Slash',
 ' Hack and slash',
 ' Hidden Object',
 ' Horror',
 ' Indie',
 ' Interactive Art',
 ' Interactive Movie',
 ' Interactive movie',
 ' MMOR',
 ' Massively Multiplayer',
 ' Mecha Simulation',
 ' Medieval / Fantasy',
 ' Moba',
 ' Multi-directional shooter',
 ' Multidirectional',
 ' Music',
 ' Music/Rhythm',
 ' On-Rails',
 ' Open World',

In [34]:
df['Genres'].value_counts()

Action                                                                       1904
Adventure                                                                    1600
Puzzle                                                                       1324
Role-Playing                                                                 1298
Action, Adventure                                                            1256
Platform                                                                     1046
Racing                                                                        893
Simulation                                                                    843
Strategy                                                                      808
Fighting                                                                      647
Visual Novel                                                                  638
Sports                                                                        516
First-Person, Sh